In [10]:
from process import *

In [11]:
a, b, c = actions('abc')

In [12]:
p = a * b * (a + b)

In [13]:
print(p)

a * b * (a + b)


In [14]:
A = Atom('A')

In [15]:
print(A)

A


In [16]:
eq_A = A == a * A
print(eq_A)

A == a * A


In [17]:
eq_A.z3expr

A == z3Seq(a, A)

In [36]:
from utils import *
from itertools import chain

class Theory(AttrDict):

    def __init__(self, **kws):
        self._variables = None
        if 'variables' in kws:
            self._variables = kws['variables']
            del kws['variables']
        
        self._atoms = None
        if 'atoms' in kws:
            self._atoms = kws['atoms']
            del kws['atoms']
                
        super().__init__(**kws)


    def variables(self):
        if self._variables is None:
            vs = []
            for eq in self.values():
                for v in chain(eq.lhs.vars(), eq.rhs.vars()):
                    if v not in vs:
                        vs.append(v)
            self._variables = vs
        return self._variables

    def var_names(self):
        '''compute list of process variables names'''
        return [v.name for v in self.variables()]


    def atoms(self):
        if self._atoms is None:
            ats = []
            for eq in self.values():
                for a in chain(eq.lhs.atoms(), eq.rhs.atoms()):
                    if a not in ats:
                        ats.append(a)
            self._atoms = ats
        return self._atoms


    def atom_names(self):
        '''compute list of process atoms names'''
        return [a.name for a in self.atoms()]

    def __str__(self):
        result = []
        vs = self.var_names()
        if len(vs)>0:
            result.append(f"# Variables\n{', '.join(vs)}")
        ats = self.atom_names()
        if len(ats)>0:
            result.append(f"# Atoms\n{', '.join(ats)}")
        result.append("# Axioms")
        result.extend(f'{ax}: {self[ax]}' for ax in self)
 
        return "\n".join(result)
        

x, y, z = vars('x, y, z')


d = dict(a = 1, b = 3)
TH1 = Theory(
    variables = vars('x, y, z'),
    atoms = atoms('zero, one'),
    AX1 = zero * x == zero,
    AX2 = one * x == x,
    AX3 = x * one == x,
    AX4 = x + zero == x,
    AX5 = x + y == y + x,
    AX6 = x + x == x,
    AX7 = (x + y) + x == x + (y + z),
    AX8 = (x * y) * z == x * (y * z),
    AX9 = (x + y) * z == x * z + y * z
    )

print([v.name for v in TH1.variables()])
print([a.name for a in TH1.atoms()])


print(TH1)

['x', 'y', 'z']
['zero', 'one']
# Variables
x, y, z,
# Atoms
zero, one,
# Axioms,
AX1: zero * x == zero,
AX2: one * x == x,
AX3: x * one == x,
AX4: x + zero == x,
AX5: x + y == y + x,
AX6: x + x == x,
AX7: x + y + x == x + (y + z),
AX8: x * y * z == x * (y * z),
AX9: (x + y) * z == x * z + y * z


In [19]:
lvars = set([v.name for eq in TH1.values() for v in eq.lhs.vars()])
rvars = set([v.name for eq in TH1.values() for v in eq.rhs.vars()])

print(lvars)
print(rvars)


{'y', 'x', 'z'}
{'y', 'x', 'z'}


In [20]:
print(TH1.AX8.z3expr)

z3Seq(z3Seq(x, y), z) == z3Seq(x, z3Seq(y, z))
